#### First: some data cleaning.

In [1]:
import pandas as pd

In [4]:
# Read in the .csv file.

Outofloop = pd.read_csv("./data/Outof.csv")

FileNotFoundError: [Errno 2] File b'./data/Outof.csv' does not exist: b'./data/Outof.csv'

In [3]:
# Check out the columns.

df.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence', 'bias',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source', 'text'],
      dtype='object')

In [4]:
# See the first five rows.

df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,...,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,...,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,...,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,...,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [5]:
# Remove all values with an "audience confidence," "bias
# confidence," or "message confidence" score below 1.

df = df[(df['audience:confidence'] >= 1) & (df['bias:confidence'] >= 1) & (df['message:confidence'] >= 1)]

In [6]:
# Remove extra columns.

df = df[['_unit_id', '_trusted_judgments', 'audience',
         'bias', 'message', 'label', 'source', 'text']]

In [7]:
# Relabel columns.

df.columns = ['unit_id', 'trusted_judgments', 'audience_feature',
              'bias_feature', 'message_feature', 'label_feature',
              'source_feature', 'text_feature']

In [8]:
# Drop NAs.

df.dropna(inplace=True)

In [9]:
# Reset index.

df.reset_index(drop=True, inplace=True)

We have social media data! This includes almost 5,000 messages on either Twitter or Facebook from various politicians. We can use the features we generated to predict things like whether the source is Twitter or Facebook, whether the bias is neutral or partisan, and so on.

In [11]:
# How many rows and columns do we have?
df.shape


(4888, 8)

In [12]:
# View first five rows.

df.head()

,unit_id,trusted_judgments,audience_feature,bias_feature,message_feature,label_feature,source_feature,text_feature
0,766192484,1,national,partisan,policy,From: Trey Radel (Representative from Florida),twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,1,national,partisan,attack,From: Mitch McConnell (Senator from Kentucky),twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,1,national,neutral,support,From: Kurt Schrader (Representative from Oregon),twitter,Please join me today in remembering our fallen...
3,766192487,1,national,neutral,policy,From: Michael Crapo (Senator from Idaho),twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,1,national,partisan,policy,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...


You may note that there are some extra symbols in the data. This is a common problem in natural language processing, especially when dealing with social media (think emoji, hashtags, links, etc.), but we're going to ignore that for now.

### Let's use Naive Bayes to predict whether a social media post was featured on Facebook or Twitter.

#### 1. Engineer a feature to turn `source_feature` into a 1/0 column, where 1 indicates `Twitter`.

In [13]:
df['twitter'] = [1 if df.loc[i,'source_feature'] == 'twitter' else 0 for i in range(df.shape[0])]

#### NOTE: Since we are solving a classification problem, what potential issue should I check for here?

In [14]:

df['twitter'].value_counts()

0    2497
1    2391
Name: twitter, dtype: int64

In [16]:
df.head()

,unit_id,trusted_judgments,audience_feature,bias_feature,message_feature,label_feature,source_feature,text_feature,twitter
0,766192484,1,national,partisan,policy,From: Trey Radel (Representative from Florida),twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,1
1,766192485,1,national,partisan,attack,From: Mitch McConnell (Senator from Kentucky),twitter,VIDEO - #Obamacare: Full of Higher Costs and ...,1
2,766192486,1,national,neutral,support,From: Kurt Schrader (Representative from Oregon),twitter,Please join me today in remembering our fallen...,1
3,766192487,1,national,neutral,policy,From: Michael Crapo (Senator from Idaho),twitter,RT @SenatorLeahy: 1st step toward Senate debat...,1
4,766192488,1,national,partisan,policy,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...,1


#### 2. Split our data into `X` and `y`.

In [18]:
X = df[['text_feature']]
y = df['twitter']

In [19]:
X.head()

,text_feature
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,VIDEO - #Obamacare: Full of Higher Costs and ...
2,Please join me today in remembering our fallen...
3,RT @SenatorLeahy: 1st step toward Senate debat...
4,.@amazon delivery #drones show need to update ...


#### 3. Split our data into training and testing sets.

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=y)

#### 4. Turn our text into features. [Documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [21]:
# Import CountVectorizer.
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate our CountVectorizer.
cvec = CountVectorizer(max_features = 500, stop_words = 'english')

In [35]:
# Fit our CountVectorizer on the training data and transform training data.
X_train_cvec = pd.DataFrame(cvec.fit_transform(X_train['text_feature']).todense(),columns = cvec.get_feature_names())

In [36]:
# Transform our testing data with the already-fit CountVectorizer.
X_test_cvec = pd.DataFrame(cvec.transform(X_test['text_feature']).todense(),
                         columns = cvec.get_feature_names())

In [37]:
X_train_cvec.head()

,00,000,10,11,12,17,1st,20,2013,2014,...,yesterday,york,young,youtube,û_,ûª,ûªm,ûªs,ûªt,ûò
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 4. Fit a Naive Bayes model!

<details><summary> Which Naive Bayes model should we pick, and why? </summary>
    
- The columns of X are all integer counts, so MultinomialNB is the best choice here.
- BernoulliNB is best when we have 0/1 counts in all columns of X. (a.k.a. dummy variables)
- GaussianNB is best when the columns of X are Normally distributed. (Practically, though, it gets used whenever BernoulliNB and MultinomialNB are inappropriate.)
</details>

In [39]:
# Import our model!

from sklearn.naive_bayes import MultinomialNB

In [40]:
# Instantiate our model!

nb = MultinomialNB()

Remember earlier that I said we had the opportunity to set priors. We could do so here if we wanted, but we'll stick with the default and allow `sklearn` to estimate priors from the training data directly.

In [41]:
# Fit our model!

model = nb.fit(X_train_cvec, y_train)

In [48]:
# Generate our predictions!

predictions = model.predict(X_test_cvec)

<details><summary> How might we evaluate our model's performance? </summary>

- Accuracy = (TP + TN) / (TP + TN + FP + FN)
- Sensitivity = TP / (TP + FN)
- Specificity = TN / (TN + FP)
- Precision = TP / (TP + FP)
- AUC ROC
</details>

<details><summary> If we have to select only one, which one should we choose? </summary>

- It depends on how exactly you define "positive" and "negative." In this case, it probably doesn't really matter - incorrectly mistaking a tweet for a Facebook post doesn't seem much better or worse than incorrectly mistaking a Facebook post for a tweet. 
- Because I believe false positives and false negatives are equally as bad, I'd probably use accuracy.
</details>

In [43]:
# Score our model on the training set.

model.score(X_train_cvec, y_train)

0.8385160938352427

In [45]:
# Score our model on the testing set.

model.score(X_test_cvec, y_test)

0.8150572831423896

<details><summary> What should we do in this case? </summary>

- Our model appears *slightly* overfit. We could:
    - try to collect more data, 
    - try using fewer features by setting `max_features` to a smaller number when instantiating our CountVectorizer,
    - try TF-IDF Vectorizer,
    - try a non-default prior **if you have subject-matter expertise**.
- Rather than regularizing, [online answers](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) suggest using a different model entirely.
- Our training performance and testing performance are pretty close, though, so there may not be a lot of changes required.
</details>

In [46]:
# Import the confusion matrix function.

from sklearn.metrics import confusion_matrix

In [49]:
# Generate a confusion matrix.

confusion_matrix(y_test, predictions)

array([[523, 101],
       [125, 473]])

In [50]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

In [51]:
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 523
False Positives: 101
False Negatives: 125
True Positives: 473


<details><summary> By default, what does a true negative mean here? </summary>

- True negatives are things we correctly predict to be negative.
- In this case, since Twitter = 1, a true negative means I correctly predict something is a Facebook post.
</details>

---

<details><summary> By default, what does a false positive mean here? </summary>

- False positives are things we falsely predict to be positive.
- In this case, since Twitter = 1, a false positive means I incorrectly preidct something is a tweet (when it's really a Facebook post).
</details>

In [52]:
X_test['text_feature']

1418           To them and to you, I say Happy Earth Day.
1039    RT @RayLaHood: New locomotive built by America...
3553    It was an honor to present Staff Sgt. Bostwick...
3946    Thanks to the folks at Strawberry Hill in Ches...
3787    Click the link below to read Senator McConnell...
                              ...                        
3609    WMIcentral.com ran this item today about our #...
4286    Just posted new tools for tax prep including w...
2254    Just ran into an old friend from the Meigs Co ...
1141    .@PattyMurray Unfortunately for RSherman_25, t...
1855    Just introduced my Manufactured Housing bill t...
Name: text_feature, Length: 1222, dtype: object